# Basic LSDB/HiPSCat Functionality

In this notebook we will demonstrate analysis of single hipscats
* querying based off of columnar information
* querying based off of conesearches
* utilize the `dask.dataframe` parallelized pandas API to concatenate methods

In [1]:
import lsdb
path_to_local_catalog = "/epyc/projects3/sam_hipscat/ADASS_demo/gaia_dr3/"

gaia = lsdb.read_hipscat(path_to_local_catalog)
gaia

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Npix,Dir
npartitions=3933,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,string,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,bool,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string,float64,float64,float64,float64,bool,bool,int64,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,floa

In [ ]:
gaia.dtypes

In [2]:
#load the catalog as a dask dataframe

gaia = lsdb.read_hipscat(path_to_local_catalog, columns=['ra', 'dec', 'pmra', 'pmdec'])
gaia

,ra,dec,pmra,pmdec
npartitions=3933,,,,
,float64,float64,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


## Lazy Evaluation and Parallelized python infrastructure
Our computations rely on the `dask.dataframe` API, which as you can see hasn't evaluated anything. For all `dask.dataframe` computations, the scheduled process aren't executed until you perform a `.compute()` at the end of the line of code. 

When utilizing this API, we can define a client to specify the number of workers, the address/port in two ways:
* with `dask.distributed` [client](https://distributed.dask.org/en/latest/client.html#dask)
* with `dask_on_ray`'s [scheduler](https://docs.ray.io/en/latest/ray-more-libs/dask-on-ray.html#scheduler)

We have personally found Dask on Ray to be better at scheduling and memory management, so we encourage others to use that.

In [3]:
from dask.distributed import Client
import ray
from ray.util.dask import enable_dask_on_ray, disable_dask_on_ray

class lsdb_client():
    def __init__(self, dask_on_ray=True, num_workers=4):
        self.dask_on_ray = dask_on_ray
        self.num_workers=num_workers
        self.start()

    def start(self):

        if self.dask_on_ray:
            self.client = ray.init(
                num_cpus=self.num_workers,
                _temp_dir="/data3/epyc/projects3/sam_hipscat"
            )
            enable_dask_on_ray()
        else:
            self.client = Client(n_workers=self.num_workers, threads_per_worker=1)

    def shutdown(self):
        if self.dask_on_ray:
            disable_dask_on_ray()
            ray.shutdown()
        else:
            self.client.close()

client = lsdb_client(dask_on_ray=True)

2023-10-30 11:22:18,703	INFO worker.py:1621 -- Started a local Ray instance.
/astro/users/sdwyatt/anaconda3/envs/lsdb_demo/lib/python3.10/site-packages/dask/config.py:694: UserWarning: Configuration key "shuffle" has been deprecated. Please use "dataframe.shuffle.algorithm" instead
  warnings.warn(


## Actually performing some computations

Now that we have our parallelized framework set up. Let's analyze our dataset with the LSDB codebase:
* performing cone searches
* querying based off of column info
* assigning columns
    * concatenating all this functionality

In [4]:
#cone search
cone_search = gaia.cone_search(
    ra=30,
    dec=30,
    radius=1
)
cone_search

,ra,dec,pmra,pmdec
npartitions=1,,,,
,float64,float64,float64,float64
,...,...,...,...


In [5]:
cone_search.compute()

,ra,dec,pmra,pmdec
_hipscat_index,,,,
599782862807367680,30.325137,29.044685,20.041401,-18.992996
599785186384674816,30.447104,29.081970,-11.465375,-21.569639
599785336708530176,30.459516,29.094229,7.987679,11.017139
599785779090161664,30.385253,29.071722,1.814819,0.799771
599785839219703808,30.343850,29.055665,-0.049741,-0.490562
...,...,...,...,...
602963402054172672,29.954635,30.971129,NaN,NaN
602963505133387776,29.972930,30.979563,0.811023,-2.701905
602963552378028032,29.993989,30.983171,1.146661,-1.118787


### Now lets perform the same computation but let's use the LSDB api to create a new column

by using the `dask.dataframe.assign` method

In [6]:
import numpy as np

gaia.cone_search(
    ra=30,
    dec=30,
    radius=10
).assign(
    pm=lambda x: np.sqrt(x['pmra']**2 + x['pmdec']**2)
).compute()

,ra,dec,pmra,pmdec,pm
_hipscat_index,,,,,
175617145399410688,36.269409,21.744470,72.164715,-119.614282,139.697253
175617381622611968,36.273256,21.750274,3.027481,-2.154353,3.715760
175620431049392128,36.345523,21.784912,2.859838,7.529109,8.053952
175620521243705344,36.313253,21.774500,6.940707,-10.674574,12.732633
175620581373247488,36.316576,21.791801,0.711730,-6.318391,6.358351
...,...,...,...,...,...
693696098010660864,25.007958,39.119437,-0.794654,0.514452,0.946644
702561447380516865,26.405785,37.503073,7.850785,-2.962412,8.391109
702561460265418752,28.066208,35.746945,8.153048,8.964197,12.117302


### Let's go a little bit further and cull our data based off of our calculation

by using the `dask.dataframe.query` method

In [7]:
gaia.cone_search(
    ra=30,
    dec=30,
    radius=15
).assign(
    pm=lambda x: np.sqrt(x['pmra']**2 + x['pmdec']**2)
).query(
    'pm > 20'
).compute()

,ra,dec,pmra,pmdec,pm
_hipscat_index,,,,,
152752195509420032,35.264570,15.807331,8.883513,-26.151785,27.619425
152752152559747072,35.266091,15.806155,8.036485,-26.123847,27.332041
152761627257602048,35.548548,15.900020,22.591434,4.112435,22.962687
152765454073462784,35.370172,15.892402,14.778710,-96.661146,97.784393
152765488433201152,35.416532,15.877083,-16.091170,-33.392061,37.066905
...,...,...,...,...,...
5603963587111419904,13.014259,28.217900,19.463394,-34.855988,39.921969
5603964261421285376,13.020839,28.242586,127.838852,-28.233709,130.919496
5604075569793728512,12.974328,28.300800,24.058023,14.078543,27.874609


### Pretty amazing results with what you can do with 4 cores and the 1.8Bn object Gaia catalog

Now lets shut down our client

In [8]:
client.shutdown()

/astro/users/sdwyatt/anaconda3/envs/lsdb_demo/lib/python3.10/site-packages/dask/config.py:694: UserWarning: Configuration key "shuffle" has been deprecated. Please use "dataframe.shuffle.algorithm" instead
  warnings.warn(
